### Interlude with Lekan (Lay $\cdot$ kon)

%% javascript MathJax.Hub.Config({ TeX: { equationNumbers: { autoNumber: "AMS" } } });

### Time-optimal control of a Double Integral Plant

We shall analytically and approximately (via the level set toolbox and the PGD scheme we describe) solve for the 

+ Terminal state of the origin of the state plane when:

  + The Target set is an entire axis

  + The Target set is a line segment

The double integral plant can represent the load in a frictionless and an environment where gravity is non-existent, for example. While this makes no engineering sense, it is useful for didactic purposes in analysing the large-scale system dynamics of interest to us.

Suppose we let $m$ be the mass (or inertial momentum) of a body, $x(t)$ the position (or angular displacement), and $\tau(t)$ the applied torque, then by Newton's second law of motion, we have 

\begin{align}
    m \ddot{x}(t) = \tau(t).
\end{align}

Let us write $u(t) = \tau(t)/m$, so that we have

\begin{align}
  \ddot{x}(t) = u(t).
\end{align}

If we consider the following change of variables,

\begin{align}
  {x}_1(t) &= x(t) \\
  {x}_2(t) &= \dot{x}(t),
\end{align}

we can then represent the system as the following system of first-order o.d.e's:

\begin{align}
  \dot{x}_1(t) &= x_2(t) \\
  \dot{x}_2(t) &= u(t).
\end{align}

Let $u(t)$ be bounded as follows $|u(t)| \le 1$ for all $t$. This is as a result of limitations on the amount of effort we want to give to our actuator. We now redefine the problem to be solved as follows:

\begin{align}
  \dot{x}_1(t) &= x_2(t) \\
  \dot{x}_2(t) &= u(t) \qquad \qquad |u(t)| \le 1.
\end{align}

The statement of the problem at hand is to _find an admissible control that forces the above system from any initial state $\left[\xi_1, \xi_2\right]$ to the origin (0,0) in the shortest possible time._

### Fillipov Differential Equation

Observe:

+ $u$ is discontinuous. 

+ This dynamical system is reminiscent of differential equations with discontinuous 
right hand sides. 

+ They have tricky solutions which are typically called Fillipov solutions.

In [1]:
import sys
import copy
import time
import cupy as cp
import numpy as np
import scipy.linalg as la
from os.path import abspath, join
import matplotlib.pyplot as plt

sys.path.append(abspath(join('..')))
from BRATVisualization.DIVisu import DoubleIntegratorVisualizer
sys.path.append(abspath(join('../..')))
from LevelSetPy.Utilities import *
from LevelSetPy.Grids import createGrid
from LevelSetPy.Helper import postTimeStepTTR
from LevelSetPy.Visualization import implicit_mesh
from LevelSetPy.DynamicalSystems import DoubleIntegrator

# POD Decomposition
from LevelSetPy.POD import *

# Chambolle-Pock for Total Variation De-Noising
from LevelSetPy.Optimization import chambollepock

# Value co-state and Lax-Friedrichs upwinding schemes
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import upwindFirstENO2
from LevelSetPy.ExplicitIntegration import artificialDissipationGLF
from LevelSetPy.ExplicitIntegration.Integration import odeCFL2, odeCFLset
from LevelSetPy.ExplicitIntegration.Term import termRestrictUpdate, termLaxFriedrichs

### Create state space and value function.

+ This would be the grid upon which we are going to control 
the car/unicycle etc -- basically anything that may have 
double integrator dynamics.

+ In addition, we'll generate the analytical solution to the time to reach the origin problem on the grid. Let's call this variable `attr`. 

+ Our initial value function will be constructed as an implicit function on the grid with a signed distance function in the geometry of a sphere with radius, $r$.

In [2]:
dint = DoubleIntegrator
u_bound = 1

In [3]:

def preprocessing():
    global dint, u_bound

    gmin = np.array(([[-1, -1]]),dtype=np.float64).T
    gmax = np.array(([[1, 1]]),dtype=np.float64).T
    g = createGrid(gmin, gmax, 101, None)

    eps_targ = 1.0
    target_rad = .2

    dint = DoubleIntegrator(g, u_bound)
    attr = dint.mttr() - target_rad
    if strcmp(args.init_cond, 'attr'):
        value_func_init = attr
    elif strcmp(args.init_cond, 'circle'):
        value_func_init = shapeSphere(g, zeros(g.dim, 1), target_rad)
    elif strcmp(args.init_cond, 'square'):
        value_func_init = shapeRectangleByCenter(g, zeros(g.dim, 1), \
                                    2*target_rad*ones(g.dim,1))

    attr = np.maximum(0, attr)

    return g, attr, value_func_init

def show_attr(g, attr):
    ### pick a bunch of initial conditions
    Delta = lambda u: u  # u is either +1 or -1
    xis = [(1,0), (.5, 0),  (0,0), (-.5, 0), (-1,0)]

    # how much timesteps to consider
    t = np.linspace(0, 1, 100)
    u = 1
    # do implicit euler integration to obtain x1 and x2
    x1p = np.empty((len(xis), g.xs[0].shape[1], len(t)))
    x2p = np.empty((len(xis), g.xs[1].shape[1], len(t)))

    # for -ve control input
    x1m = np.empty((len(xis), g.xs[0].shape[1], len(t)))
    x2m = np.empty((len(xis), g.xs[1].shape[1], len(t)))

    for i in range(len(xis)):
        for k in range(len(t)):
            x2p[i, :,k] = xis[i][1] + Delta(u) * t[k]
            x1p[i, :,k] = xis[i][0] + .5 * Delta(u) * x2p[i,:,k]**2 - .5 * Delta(u) * xis[i][1]**2
            x2m[i, :,k] = xis[i][1] + Delta(-u) * t[k]
            x1m[i, :,k] = xis[i][0] + .5 * Delta(-u) * x2m[i,:,k]**2 - .5 * Delta(-u) * xis[i][1]**2

    plt.ion()
    fig, [ax1,ax2] = plt.subplots(1, 2, figsize=(12,4))

    # Plot a few snapshots.
    color = iter(plt.cm.viridis(np.linspace(.25, 1, 5)))
    for init_cond in range(len(xis)):
        x1_temp = x1p[init_cond, -1, :]
        x2_temp = x2p[init_cond, -1, :]
        color_now=next(color)
        ax1.grid('on')
        ax1.plot(x1_temp, x2_temp, color=color_now)

        x1_temp = x1m[init_cond, -1, :]
        x2_temp = x2m[init_cond, -1, :]
        ax1.plot(x1_temp, x2_temp, color=color_now, label=rf"$x_{{{xis[init_cond]}}}$")
    #ax1.set_xlim(-1, 1)
    #ax1.set_ylim(-1, 1)
    ax1.set_xlabel(r"Velocity ($m/s$)")
    ax1.set_ylabel(r"Time $s$")
    ax1.set_title(r"Forced Trajectories.")
    ax1.legend(loc="lower right", fontsize=8, bbox_to_anchor=(1.01,.05))


    # Plot all vectograms(snapshots) in space and time.
    cdata = ax2.pcolormesh(g.xs[0], g.xs[1], attr, shading="nearest", cmap="magma")
    plt.colorbar(cdata, ax=ax2, extend="both")
    ax2.set_xlabel(r"Position($m$)")
    ax2.set_ylabel(r"Velocity $m/s$")
    ax2.set_title(r"State Plane")

    fig.suptitle("Isochrones for the Double Integral Plant.")

    fig.canvas.draw()
    fig.canvas.flush_events()
    time.sleep(args.pause_time)

def show_init_levels(g, attr, value_func_init):

    f, (ax1, ax2) = plt.subplots(1,2,figsize=(12, 4))

    ax1.contour(g.xs[0], g.xs[1], attr, colors='red')
    ax1.set_title('Analytical TTR')
    ax1.set_xlabel(r"$x_1 (m)$")
    ax1.set_ylabel(r"$x_2 (ms^{-1})$")
    ax1.set_xlim([-1.02, 1.02])
    ax1.set_ylim([-1.01, 1.01])
    ax1.grid()

    ax2.contour(g.xs[0], g.xs[1], value_func_init, colors='blue')
    ax2.set_title('Numerical TTR')
    ax2.set_xlabel(r"$x_1 (m)$")
    ax2.set_xlim([-1.02, 1.02])
    ax2.set_ylim([-1.01, 1.01])
    ax2.grid()

    f.suptitle(f"Levelsets")

    f.canvas.draw()
    f.canvas.flush_events()
    time.sleep(args.pause_time)


#### Trajectories in the $x_1$-$x_2$ plane

\begin{align}
x_1 &= \xi_1 + \frac{1}{2} \Delta x_2^2 - \frac{1}{2} \Delta \xi_2^2, \text{ where } \quad t = \Delta(x_2 - \xi_2)  \\
x_2 &= \xi_2 + \Delta t
\end{align}

where $\Delta= u \equiv \pm 1$, 

$t$ is the time it takes to arrive at our goal; 

and $x_1$ and $x_2$ are the position and velocity of the body respectively.

In [5]:
g, attr, value_func_init = preprocessing()

### pick a bunch of initial conditions
Delta = lambda u: u  # u is either +1 or -1
xis = [(1,0), (.5, 0),  (0,0), (-.5, 0), (-1,0)]

# how much timesteps to consider
t = np.linspace(-1, 1, 100)
u = 1
# do implicit euler integration to obtain x1 and x2
x1p = np.empty((len(xis), g.xs[0].shape[1], len(t)))
x2p = np.empty((len(xis), g.xs[1].shape[1], len(t)))
# states under negative control law
x1m  = np.empty((len(xis), g.xs[0].shape[1], len(t)))
x2m = np.empty((len(xis), g.xs[1].shape[1], len(t)))

plt.ion()
fig, [ax1,ax2] = plt.subplots(1, 2, figsize=(12,4))

for i in range(len(xis)):
    for k in range(len(t)):
        x2p[i, :,k] = xis[i][1] + Delta(u) * t[k]
        x1p[i, :,k] = xis[i][0] + .5 * Delta(u) * x2p[i,:,k]**2 - .5 * Delta(u) * xis[i][1]**2
        # state trajos under -ve control law
        x2m[i, :,k] = xis[i][1] + Delta(-u) * t[k]
        x1m[i, :,k] = xis[i][0] + .5 * Delta(-u) * x2p[i,:,k]**2 - .5 * Delta(-u) * xis[i][1]**2


# Plot a few snapshots for different initial conditions.
color = iter(plt.cm.viridis(np.linspace(.25, 1, 2*len(xis))))
for init_cond in range(len(xis)):
    # state trajectories are unique for every initial cond
    # so pick last state
    ax1.plot(x1p[init_cond, -1, :], x2p[init_cond, -1, :], color=next(color))
    ax1.plot(x1m[init_cond, -1, :], x2m[init_cond, -1, :], color=next(color))
    ax1.grid('on')

#ax1.set_xlim(-1, 1)
#ax1.set_ylim(-1, 1)
ax1.set_xlabel(r"Linear Speed ($m/s$)")
ax1.set_ylabel(r"Torque $\bm{u}$")
ax1.set_title(r"Forced Trajectories.")
ax1.legend(loc="lower right", fontsize=8, bbox_to_anchor=(1.01,.05))


# Plot all vectograms(snapshots) in space and time.
cdata = ax2.pcolormesh(g.xs[0], g.xs[1], attr, shading="nearest", cmap="magma")
plt.colorbar(cdata, ax=ax2, extend="both")
ax2.set_xlabel(r"Position($m$)")
ax2.set_ylabel(r"Velocity $m/s$")
ax2.set_title(r"State Plane")

fig.suptitle("Isochrones for the Double Integral Plant.")

fig.canvas.draw()
fig.canvas.flush_events()
time.sleep(args.pause_time)


NameError: name 'args' is not defined

### Examining the Level Sets
The analytical time-to-reach is resolved as

\begin{align}
  &\left[\left(x_2 + \sqrt{4 x_1 + 2 x_2^2}\right) \, \cdot \, \left(x_1 > \Gamma\right)\right] + 
            &\left[\left(-x_2 + \sqrt{2x_2^2 - 4 x_1}\right)  \, \cdot \,  (x_1 < \Gamma) \right] +
            |x_2|  \, \cdot \,  \left(x_1 = \Gamma\right).
\end{align}

where $\Gamma$ is the switching curve defined as 

\begin{align}
    \Gamma = -\dfrac{1}{2} \cdot x_2 \dot |x_2|
\end{align}

### Optimize the trajectories

In [ ]:
# extrapolate the data onto a bigger grid by total variation with ADMM
value_scaled = Ur @ value_rob @ Ur.T

In [ ]:

### pick a bunch of initial conditions
Delta = lambda u: u  # u is either +1 or -1
xis = [(1,0), (.5, 0),  (0,0), (-.5, 0), (-1,0)]

# how much timesteps to consider
t = np.linspace(0, 1, 100)
u = 1
# do implicit euler integration xto obtain x1 and x2
x1p = np.empty((len(xis), g.xs[0].shape[1], len(t)))
x2p = np.empty((len(xis), g.xs[1].shape[1], len(t)))

# for -ve control input
x1m = np.empty((len(xis), g.xs[0].shape[1], len(t)))
x2m = np.empty((len(xis), g.xs[1].shape[1], len(t)))

for i in range(len(xis)):
    for k in range(len(t)):
        x2p[i, :,k] = xis[i][1] + Delta(u) * t[k]
        x1p[i, :,k] = xis[i][0] + .5 * Delta(u) * x2p[i,:,k]**2 - .5 * Delta(u) * xis[i][1]**2
        x2m[i, :,k] = xis[i][1] + Delta(-u) * t[k]
        x1m[i, :,k] = xis[i][0] + .5 * Delta(-u) * x2m[i,:,k]**2 - .5 * Delta(-u) * xis[i][1]**2

plt.ion()
fig, [ax1,ax2] = plt.subplots(1, 2, figsize=(12,4))

# Plot a few snapshots.
color = iter(plt.cm.viridis(np.linspace(.25, 1, 5)))
for init_cond in range(len(xis)):
    x1_temp = x1p[init_cond, -1, :]
    x2_temp = x2p[init_cond, -1, :]
    color_now=next(color)
    ax1.grid('on')
    ax1.plot(x1_temp, x2_temp, color=color_now)

    x1_temp = x1m[init_cond, -1, :]
    x2_temp = x2m[init_cond, -1, :]
    ax1.plot(x1_temp, x2_temp, color=color_now, label=rf"$x_{{{xis[init_cond]}}}$")
#ax1.set_xlim(-1, 1)
#ax1.set_ylim(-1, 1)
ax1.set_xlabel(r"Velocity ($m/s$)")
ax1.set_ylabel(r"Time $s$")
ax1.set_title(r"Forced Trajectories.")
ax1.legend(loc="lower right", fontsize=8, bbox_to_anchor=(1.01,.05))


# Plot all vectograms(snapshots) in space and time.
cdata = ax2.pcolormesh(g.xs[0], g.xs[1], attr, shading="nearest", cmap="magma")
plt.colorbar(cdata, ax=ax2, extend="both")
ax2.set_xlabel(r"Position($m$)")
ax2.set_ylabel(r"Velocity $m/s$")
ax2.set_title(r"State Plane")

fig.suptitle("Isochrones for the Double Integral Plant.")

fig.canvas.draw()
fig.canvas.flush_events()
time.sleep(args.pause_time)


In [ ]:
oddIndices = cell(g.dim)
evenIndices = cell(g.dim)
for i in range(g.dim):
    evenIndices = np.arange(0, value_scaled.shape[i], min_rank, dtype=np.intp)
    oddIndices = np.arange(1, value_scaled.shape[i], min_rank, dtype=np.intp)

# interpolate data onto new value structure for ease of maniupulation
value_scaled[np.ix_(evenIndices, evenIndices)] = value_rob[:-1,:-1]
value_scaled[np.ix_(oddIndices, oddIndices)] = value_rob[2:,2:]

value_rob = cp.asarray(value_scaled)
del value_scaled

# Visualization paramters
spacing = tuple(g.dx.flatten().tolist())
#turn the state space over to the gpu
g.xs = [cp.asarray(x) for x in g.xs]

In [ ]:
global dint, u_bound
#turn the state space over to the gpu
g.xs = [cp.asarray(x) for x in g.xs]
finite_diff_data = Bundle(dict(innerFunc = termLaxFriedrichs,
            innerData = Bundle({'grid':g,
                'hamFunc': dint.hamiltonian,
                'partialFunc': dint.dynamics,
                'dissFunc': artificialDissipationGLF,
                'derivFunc': upwindFirstENO2,
                }),
                positive = False,  # direction to grow the updated level set
            ))

small = 100*eps
t_span = np.linspace(0, 2.0, 20)
options = Bundle(dict(factorCFL=0.75, stats='on', maxStep=realmax, 						singleStep='off', postTimestep=postTimeStepTTR))

y = copy.copy(value_func_init.flatten())
y, finite_diff_data = postTimeStepTTR(0, y, finite_diff_data)
value_func = cp.asarray(copy.copy(y.reshape(g.shape)))

In [ ]:

    params = Bundle(
    					{'grid': g,
    					#'pgd_grid': g,
    					'disp': True,
    					'labelsize': 16,
    					'labels': "Initial 0-LevelSet",
    					'linewidth': 2,
    					'elevation': args.elevation,
    					'azimuth': args.azimuth,
    					'mesh': value_func.get(),
    					'pgd_mesh': value_rob.get(),
    					'init_conditions': False,
    					'pause_time': args.pause_time,
    					'level': 0, # which level set to visualize
    					'winsize': (16,9),
    					'fontdict': Bundle({'fontsize':12, 'fontweight':'bold'}),
    					"savedict": Bundle({"save": False,
    								"savename": "rcbrt",
    								"savepath": "../jpeg_dumps/rcbrt"})
    					})

    if args.visualize:
    	viz = DoubleIntegratorVisualizer(params)

    value_func_all = np.zeros((len(t_span),)+value_func.shape)
    value_pgd_all = np.zeros((len(t_span),)+value_rob.shape)

    cur_time, max_time = 0, t_span[-1]
    step_time = (t_span[-1]-t_span[0])/8.0

    start_time = cputime()
    itr_start = cp.cuda.Event()
    itr_end = cp.cuda.Event()

    while max_time-cur_time > small * max_time:
    	itr_start.record()
    	cpu_start = cputime()

    	time_step = f"{cur_time:.2f}/{max_time:.2f}"

    	y0 = value_func.flatten()
    	y0_rob = cp.asarray(value_rob.flatten())

    	#How far to integrate
    	t_span = np.hstack([cur_time, min(max_time, cur_time + step_time)])

    	# advance one step of integration
    	t, y, finite_diff_data = odeCFL2(termRestrictUpdate, t_span, y0, options, finite_diff_data)
    	t_rob, y_rob, finite_diff_rob = odeCFL2(termRestrictUpdate, t_span, y0_rob, options, finite_diff_data)
    	cp.cuda.Stream.null.synchronize()
    	cur_time = t if np.isscalar(t) else t[-1]

    	value_func = cp.reshape(y, g.shape)
    	value_rob = cp.reshape(y_rob, g.shape)

    	if args.visualize:
    		ls_mesh = value_func.get()
    		pgd_mesh = value_rob.get()
    		viz.update_tube(attr, ls_mesh, pgd_mesh, cur_time, delete_last_plot=True)

    	itr_end.record()
    	itr_end.synchronize()
    	cpu_end = cputime()

    	print(f't: {time_step} | GPU time: {(cp.cuda.get_elapsed_time(itr_start, itr_end)):.2f} | CPU Time: {(cpu_end-cpu_start):.2f}')
    	print(f'Bnds {min(y):.2f}/{max(y):.2f} | PGD Bnds {min(y_rob):.2f}/{max(y_rob):.2f} | Norm: {np.linalg.norm(y, 2):.2f} | PGD Norm: {np.linalg.norm(y_rob, 2):.2f} ')

    	# store this brt
    	value_func_all[cur_time] = ls_mesh
    	value_pgd_all[cur_time] = pgd_mesh

    end_time = cputime()
    print(f'Total BRS/BRT execution time {(end_time - start_time):.4f} seconds.')



In [ ]:
args = Bundle(dict(elevation=5, azimuth=5, pause_time=1,visualize=True))
#turn the state space over to the gpu
g.xs = [cp.asarray(x) for x in g.xs]
finite_diff_data = Bundle(dict(innerFunc = termLaxFriedrichs,
			innerData = Bundle({'grid':g, 'hamFunc': dint.hamiltonian,
				'partialFunc': dint.dynamics,
				'dissFunc': artificialDissipationGLF,
				'derivFunc': upwindFirstENO2,
				'input_bound': u_bound,
				'innerFunc': termLaxFriedrichs,
				}),
				positive = False,  # direction to grow the updated level set
			))
small = 100*eps
t_span = np.linspace(0, 2.0, 20)
options = Bundle(dict(factorCFL=0.75, stats='on', maxStep=realmax, \
						singleStep='off', postTimestep=postTimeStepTTR))

y = copy.copy(value_func.flatten())
y, finite_diff_data = postTimeStepTTR(0, y, finite_diff_data)
value_func = cp.asarray(copy.copy(y.reshape(g.shape)))

In [ ]:
value0_cpu = value_func.get()
U, Sigma, V = la.svd(value0_cpu, full_matrices=False)

# sanity checks
print(U.shape, Sigma.shape, V.shape)
np.allclose(U@np.diag(Sigma)@V, value0_cpu)

### Project Value Function Onto Reduced Basis
To project in an informed manner, we choose an approximation error
    $\epsilon$ e.g. 1e-5. 

We can construct a POD basis $V_r$ to use in the construction of the ROM.
There are a few ways to make an informed choice of $r$; in this example, we examine the _projection error_, defined by
$$
\text{err}_\text{projection} = \frac{||X - V_rV_r^\mathsf{T}X||_F}{||X||_F}.
$$
We choose $r$ so that the projection error is under $10^{-5}$.

In [ ]:
# informed reduced basis rank
min_rank = minimal_projection_error(value0_cpu, U, eps=1e-5, plot=True)
print(min_rank)

In [ ]:
Ur, Sigmar, Vr = U[:,:min_rank], Sigma[:min_rank], V[:,:min_rank]
# be sure the singular values are arranged in descending order
plt.plot(Sigmar)
plt.grid('on')
plt.title('Singular values from DMD')
plt.show()
# project value function in reduced basis
value_rob = Ur.T @ value0_cpu @ Ur
# value_rob = value0_cpu @ Ur
print(f'Value function in ROB: {value_rob.shape}')

### Interpolate the reduced value function onto the big grid

In [ ]:
value_scaled = np.zeros_like(value_func_init)

oddIndices = cell(g.dim)
evenIndices = cell(g.dim)
for i in range(g.dim):
    evenIndices = np.arange(0, value_scaled.shape[i], 2, dtype=np.intp)
    oddIndices = np.arange(1, value_scaled.shape[i], 2, dtype=np.intp)

# interpolate data onto new value structure for ease of maniupulation
value_scaled[np.ix_(evenIndices, evenIndices)] = value_rob[:-1,:-1]
value_scaled[np.ix_(oddIndices, oddIndices)] = value_rob[2:,2:]

value_rob = cp.asarray(value_scaled)
del value_scaled

In [ ]:
# Visualization paramters
spacing = tuple(g.dx.flatten().tolist())
#turn the state space over to the gpu
g.xs = [cp.asarray(x) for x in g.xs]

params = Bundle(
					{'grid': g,
					#'pgd_grid': g,
					'disp': True,
					'labelsize': 16,
					'labels': "Initial 0-LevelSet",
					'linewidth': 2,
					'elevation': args.elevation,
					'azimuth': args.azimuth,
					'mesh': value_func.get(),
					'pgd_mesh': value_rob.get(),
					'init_conditions': False,
					'pause_time': args.pause_time,
					'level': 0, # which level set to visualize
					'winsize': (16,9),
					'fontdict': Bundle({'fontsize':12, 'fontweight':'bold'}),
					"savedict": Bundle({"save": False,
								"savename": "rcbrt",
								"savepath": "../jpeg_dumps/rcbrt"})
					})

if args.visualize:
	viz = DoubleIntegratorVisualizer(params)

value_func_all = [value_func]

cur_time, max_time = 0, t_span[-1]
step_time = (t_span[-1]-t_span[0])/8.0

start_time = cputime()
itr_start = cp.cuda.Event()
itr_end = cp.cuda.Event()

+ Interpolate V_rob onto V_max

+ Look into migrate_grid from helperOC

+ Auto Encoder to project orig grid onto reduced grids

+ 

In [ ]:
while max_time-cur_time > small * max_time:
	itr_start.record()
	cpu_start = cputime()

	time_step = f"{cur_time:.2f}/{max_time:.2f}"

	y0 = value_func.flatten()
	y0_rob = cp.asarray(value_rob.flatten())

	#How far to integrate
	t_span = np.hstack([cur_time, min(max_time, cur_time + step_time)])

	# advance one step of integration
	t, y, finite_diff_data = odeCFL2(termRestrictUpdate, t_span, y0, options, finite_diff_data)
	t_rob, y_rob, finite_diff_rob = odeCFL2(termRestrictUpdate, t_span, y0_rob, options, finite_diff_data)
	cp.cuda.Stream.null.synchronize()
	cur_time = t if np.isscalar(t) else t[-1]

	value_func = cp.reshape(y, g.shape)
	value_rob = cp.reshape(y_rob, g.shape)

	if args.visualize:
		ls_mesh = value_func.get()
		pgd_mesh = value_rob.get()
		viz.update_tube(attr, ls_mesh, pgd_mesh, cur_time, False)
	
	itr_end.record()
	itr_end.synchronize()
	cpu_end = cputime()
	
	print(f't: {time_step} | GPU time: {(cp.cuda.get_elapsed_time(itr_start, itr_end)):.2f} | CPU Time: {(cpu_end-cpu_start):.2f}')
	print(f'Bnds {min(y):.2f}/{max(y):.2f} | PGD Bnds {min(y_rob):.2f}/{max(y_rob):.2f} | Norm: {np.linalg.norm(y, 2):.2f} | PGD Norm: {np.linalg.norm(y_rob, 2):.2f} ')
	
	# store this brt
	value_func_all.append(ls_mesh)
	value_func_all.append(pgd_mesh)
		
end_time = cputime()
print(f'Total BRS/BRT execution time {(end_time - start_time):.4f} seconds.')